In [4]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from scipy import stats
import statsmodels.api as smf

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)
plt.style.use('ggplot')

In [110]:
# Read raw monthly crsp file
data = pd.read_csv('Data.csv', parse_dates = [1], low_memory = False)
data.head()

,PERMNO,date,SHRCD,EXCHCD,SICCD,COMNAM,PRC,RET,SHROUT,RETX
0,10001,1990-01-31,11.0,3.0,4920,GREAT FALLS GAS CO,-9.9375,-0.018519,1022.0,-0.018519
1,10001,1990-02-28,11.0,3.0,4920,GREAT FALLS GAS CO,-9.8750,-0.006289,1022.0,-0.006289
2,10001,1990-03-30,11.0,3.0,4920,GREAT FALLS GAS CO,-9.8750,0.012658,1027.0,0.000000
3,10001,1990-04-30,11.0,3.0,4920,GREAT FALLS GAS CO,-9.8750,0.000000,1027.0,0.000000
4,10001,1990-05-31,11.0,3.0,4920,GREAT FALLS GAS CO,9.7500,-0.012658,1027.0,-0.012658


In [111]:
# only ordinary shares
data = data.query('SHRCD==10 or SHRCD==11')
 
# only traded on NYSE, NASDAQ, AMEX
data = data.query('EXCHCD==1 or EXCHCD==2 or EXCHCD==3')

# drop financials
mask_z = data['SICCD'] == 'Z'
data['SICCD'] = data['SICCD'][-mask_z]
data['SICCD'] = data['SICCD'].dropna().astype(int)
data = data.query('SICCD < 6000 or SICCD > 6999')

# Clean return taking out strings 
data['RET'] = data['RET'].replace(['C','B'],np.nan)

# Keep value above -50 to avoid any errors 
data['RET'] = data['RET'].astype('float')
mask_ret = data['RET'] > -50
data = data[mask_ret]

# PRC is the closing price or the negative bid/ask average. We need to replace negative values by absolute values
data['PRC'] = data['PRC'].abs()

# Drop observations with prices below 5$
data.loc[(data['PRC'] < 5),'RET'] = np.nan

# Create month and year variables
data['year'] = pd.DatetimeIndex(data['date']).year
data['month'] = pd.DatetimeIndex(data['date']).month

# Create Market Cap variable
data['CAP'] = data['PRC']*data['SHROUT']

# Drop duplicates
data = data.drop_duplicates(subset = ['PERMNO', 'date'], keep = 'first')

# Clean and reorder
data.drop(['SHRCD','EXCHCD','SICCD','SHROUT'], axis=1, inplace = True)
data = data[['PERMNO', 'date', 'year', 'month', 'COMNAM', 'PRC','CAP', 'RETX', 'RET']]
data[data['PERMNO']==10010]


,PERMNO,date,year,month,COMNAM,PRC,CAP,RETX,RET
842,10010,1990-01-31,1990,1,CABOT MEDICAL CORP,2.2500,15014.2500,-0.162791,NaN
843,10010,1990-02-28,1990,2,CABOT MEDICAL CORP,2.8750,19184.8750,0.277778,NaN
844,10010,1990-03-30,1990,3,CABOT MEDICAL CORP,2.8750,19184.8750,0.000000,NaN
845,10010,1990-04-30,1990,4,CABOT MEDICAL CORP,2.7500,18301.2500,-0.043478,NaN
846,10010,1990-05-31,1990,5,CABOT MEDICAL CORP,5.1250,34106.8750,0.863636,0.863636
847,10010,1990-06-29,1990,6,CABOT MEDICAL CORP,10.0000,66550.0000,0.951219,0.951219
848,10010,1990-07-31,1990,7,CABOT MEDICAL CORP,12.5000,84512.5000,0.250000,0.250000
849,10010,1990-08-31,1990,8,CABOT MEDICAL CORP,9.3750,63384.3750,-0.250000,-0.250000
850,10010,1990-09-28,1990,9,CABOT MEDICAL CORP,8.5000,57468.5000,-0.093333,-0.093333
851,10010,1990-10-31,1990,10,CABOT MEDICAL CORP,7.2500,50742.7500,-0.147059,-0.147059


In [113]:
data['std_36'] = data.groupby('PERMNO')['RET'].rolling(36, min_periods = 36).std().reset_index(0,drop=True)
data[data['PERMNO']==10001]

,PERMNO,date,year,month,COMNAM,PRC,CAP,RETX,RET,std_36
0,10001,1990-01-31,1990,1,GREAT FALLS GAS CO,9.93750,10156.12500,-0.018519,-0.018519,NaN
1,10001,1990-02-28,1990,2,GREAT FALLS GAS CO,9.87500,10092.25000,-0.006289,-0.006289,NaN
2,10001,1990-03-30,1990,3,GREAT FALLS GAS CO,9.87500,10141.62500,0.000000,0.012658,NaN
3,10001,1990-04-30,1990,4,GREAT FALLS GAS CO,9.87500,10141.62500,0.000000,0.000000,NaN
4,10001,1990-05-31,1990,5,GREAT FALLS GAS CO,9.75000,10013.25000,-0.012658,-0.012658,NaN
5,10001,1990-06-29,1990,6,GREAT FALLS GAS CO,9.75000,10052.25000,0.000000,0.014103,NaN
6,10001,1990-07-31,1990,7,GREAT FALLS GAS CO,10.00000,10310.00000,0.025641,0.025641,NaN
7,10001,1990-08-31,1990,8,GREAT FALLS GAS CO,9.50000,9794.50000,-0.050000,-0.050000,NaN
8,10001,1990-09-28,1990,9,GREAT FALLS GAS CO,9.75000,10179.00000,0.026316,0.040789,NaN
9,10001,1990-10-31,1990,10,GREAT FALLS GAS CO,9.62500,10048.50000,-0.012821,-0.012821,NaN


In [116]:
data = data.dropna(axis=0, subset=['std_36'])
data['decile'] = data.groupby('date')['std_36'].transform(lambda x: pd.qcut(x, 10, labels=False))
data['decile'] = 1 + data['decile'].astype(int)
data[data['PERMNO']==11378]

,PERMNO,date,year,month,COMNAM,PRC,CAP,RETX,RET,std_36,decile
107860,11378,1992-12-31,1992,12,CALGON CARBON CORP,17.6250,7.204042e+05,-0.078431,-0.076340,0.109223,7
107861,11378,1993-01-29,1993,1,CALGON CARBON CORP,17.3750,7.101858e+05,-0.014184,-0.014184,0.105793,6
107862,11378,1993-02-26,1993,2,CALGON CARBON CORP,17.6250,7.204042e+05,0.014388,0.014388,0.102606,6
107863,11378,1993-03-31,1993,3,CALGON CARBON CORP,17.0000,6.948580e+05,-0.035461,-0.033191,0.102133,6
107864,11378,1993-04-30,1993,4,CALGON CARBON CORP,17.0000,6.948580e+05,0.000000,0.000000,0.101956,6
107865,11378,1993-05-28,1993,5,CALGON CARBON CORP,15.8750,6.488748e+05,-0.066176,-0.066176,0.102496,6
107866,11378,1993-06-30,1993,6,CALGON CARBON CORP,12.0000,4.923480e+05,-0.244094,-0.241575,0.109826,7
107867,11378,1993-07-30,1993,7,CALGON CARBON CORP,11.8750,4.872194e+05,-0.010417,-0.010417,0.109521,7
107868,11378,1993-08-31,1993,8,CALGON CARBON CORP,11.7500,4.820908e+05,-0.010526,-0.010526,0.107336,7
107869,11378,1993-09-30,1993,9,CALGON CARBON CORP,11.2500,4.616888e+05,-0.042553,-0.039149,0.105611,7


In [115]:
data.groupby('decile') ['RET'].mean()

decile
1     0.009193
2     0.009911
3     0.009747
4     0.010034
5     0.010270
6     0.011520
7     0.012559
8     0.012679
9     0.013599
10    0.019631
Name: RET, dtype: float64